In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My Drive/Elianna/HousePrice_Kaggle/Kaggle-House-Price-ML

/content/gdrive/My Drive/Elianna/HousePrice_Kaggle/Kaggle-House-Price-ML


In [58]:
#libraries we need
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
pd.options.mode.chained_assignment = None
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [61]:
SEED                    = 12345   # global random seed for better reproducibility
GLM_SELECTION_THRESHOLD = 0.001

np.random.seed(SEED)                     # set random seed for better reproducibility
h2o.init(max_mem_size='24G', nthreads=4) # start h2o with plenty of memory and threads
h2o.remove_all()                         # clears h2o memory
h2o.no_progress()                        # turn off h2o progress indicators    

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpy7nwfulp
  JVM stdout: /tmp/tmpy7nwfulp/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpy7nwfulp/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.5
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_unknownUser_p2dipb
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,24 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [87]:
train_df = pd.read_csv('processed-data/new_train.csv')
test_df = pd.read_csv('processed-data/new_test.csv')

In [88]:
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)

In [89]:
train_y = train_df['SalePrice']
train_x = train_df.drop('SalePrice', axis = 1)

In [90]:
import h2o
from sklearn.model_selection import train_test_split
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [91]:
# 70% train, 30% valid
X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.30, random_state=0)

In [115]:
y_name = 'SalePrice'
x_names = X_train.columns

In [116]:
x_names

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape',
       'LandSlope', 'OverallQual', 'OverallCond', 'MasVnrArea',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=224)

In [110]:
train = pd.concat([X_train, y_train], axis=1)
valid = pd.concat([X_valid, y_valid], axis=1)
train_hf = h2o.H2OFrame(train)
valid_hf = h2o.H2OFrame(valid)
# glm = H2OGeneralizedLinearEstimator(alpha=.25, lambda_search=True)
glm = H2OGeneralizedLinearEstimator()
glm.train(x = x_names, y = y_name, training_frame = train_hf)

H2OTypeError: ignored

In [108]:
train['SalePrice']

64      219500.0
682     173000.0
960     116500.0
1384    105000.0
1100     60000.0
          ...   
763     337000.0
835     128000.0
1216    112000.0
559     234000.0
684     221000.0
Name: SalePrice, Length: 1022, dtype: float64